In [2]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

## Constants

In [3]:
PATH = "C:/Users/Mesh/Desktop/PythonFinance/Python4Finance/BigSectorStocks/"

# Start end date defaults
S_DATE = "2018-02-01"
E_DATE = "2023-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

## Get Dataframe from CSV

In [4]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_stock_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

## Plot Ichimoku

In [5]:
# Used to generate the red and green fill for the Ichimoku cloud
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

In [6]:
def get_Ichimoku(df):

    candle = go.Candlestick(x=df.index, open=df['Open'],
    high=df['High'], low=df["Low"], close=df['Close'], name="Candlestick")

    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0)
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()

    df = df.groupby('group')

    dfs = []
    for name, data in df:
        dfs.append(data)

    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y=df.SpanA,
        line=dict(color='rgba(0,0,0,0)')))

        fig.add_traces(go.Scatter(x=df.index, y=df.SpanB,
        line=dict(color='rgba(0,0,0,0)'),
        fill='tonexty',
        fillcolor=get_fill_color(df['label'].iloc[0])))

    baseline = go.Scatter(x=df1.index, y=df1['Baseline'], 
    line=dict(color='pink', width=2), name="Baseline")

    conversion = go.Scatter(x=df1.index, y=df1['Conversion'], 
    line=dict(color='black', width=1), name="Conversion")

    lagging = go.Scatter(x=df1.index, y=df1['Lagging'], 
    line=dict(color='purple', width=2), name="Lagging")

    span_a = go.Scatter(x=df1.index, y=df1['SpanA'], 
    line=dict(color='green', width=2, dash='dot'), name="Span A")

    span_b = go.Scatter(x=df1.index, y=df1['SpanB'], 
    line=dict(color='red', width=1, dash='dot'), name="Span B")

    fig.add_trace(candle)
    fig.add_trace(baseline)
    fig.add_trace(conversion)
    fig.add_trace(lagging)
    fig.add_trace(span_a)
    fig.add_trace(span_b)
    
    fig.update_layout(height=1200, width=1800, showlegend=True)

    fig.show()

## Get Sector Stocks

In [7]:
sec_df = pd.read_csv("C:/Users/Mesh/Desktop/PythonFinance/Python4Finance/nasdaq_screener.csv")

indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Telecommunications"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Finance"]
material_df = sec_df.loc[sec_df['Sector'] == "Basic Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

## Returns a DF with Cumulative Return for all Stocks

In [8]:
def get_cum_ret_for_stocks(stock_df):
    tickers = []
    cum_rets = []

    for index, row in stock_df.iterrows():
        df = get_stock_df_from_csv(row['Symbol'])
        if df is None:
            pass
        else:
            tickers.append(row['Symbol'])
            # print(row['Symbol'])
            cum = df['cum_return'].iloc[-1]
            cum_rets.append(cum)
    return pd.DataFrame({'Symbol':tickers, 'CUM_RET':cum_rets})

In [9]:
finance = get_cum_ret_for_stocks(financial_df)
industrial = get_cum_ret_for_stocks(indus_df)
health_care = get_cum_ret_for_stocks(health_df)
it = get_cum_ret_for_stocks(it_df)
commun = get_cum_ret_for_stocks(comm_df)
staple = get_cum_ret_for_stocks(staple_df)
discretion = get_cum_ret_for_stocks(discretion_df)
utility = get_cum_ret_for_stocks(utility_df)
material = get_cum_ret_for_stocks(material_df)
restate = get_cum_ret_for_stocks(restate_df)
energy = get_cum_ret_for_stocks(energy_df)

File Doesn't Exist


## Top Industrials

In [10]:
top_industry_df = industrial.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_industry_df

,Symbol,CUM_RET
874,SGML,20.114632
1041,WFRD,7.857143
31,AEHR,7.832776
64,AMRC,7.098824
636,LWLG,5.292683
845,RUN,4.803954
1030,VRTV,4.680977
592,KNW,4.478261
995,UAN,4.260726
582,KEYS,4.038328


In [ ]:
# SGML, PWR
for ticker in top_industry_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Healthcare

In [75]:
top_health_df = health_care.sort_values(by=['CUM_RET'], ascending=False).head(30).tail(20).copy()
top_health_df

,Symbol,CUM_RET
687,KRYS,8.744797
481,FLGT,8.717877
228,BLU,8.651637
978,PRPH,8.216714
408,DXCM,7.858098
561,HROW,7.841463
648,ISEE,7.696551
575,HZNP,7.580926
1157,SWAV,6.949337
131,ARGX,6.645022


In [18]:
# DXCM, LLY, CYTK
for ticker in top_health_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

NameError: name 'top_health_df' is not defined

## Top Information Technology

In [78]:
top_tech_df = it.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_tech_df

,Symbol,CUM_RET
260,ENPH,118.511727
241,DTST,24.857143
2,ACCD,15.381818
323,GMGI,13.666666
433,LSCC,11.875639
638,RENN,11.495069
129,CALX,10.854264
776,TTD,9.267069
580,PERI,8.748299
50,APPS,8.450000


In [ ]:
# ENPH, LSCC, RENN, NSSC
for ticker in top_tech_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Communications

In [81]:
top_comm_df = commun.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_comm_df

,Symbol,CUM_RET
9,AVNW,4.049771
20,IDT,2.692752
55,TMUS,2.242949
31,LTRX,2.139175
17,EXTR,1.509724
16,ESE,1.446161
15,CSCO,1.445556
42,SJR,1.401234
13,CHT,1.247467
18,HPE,1.236098


In [ ]:
# EXTR, PTNR
for ticker in top_comm_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Staples

In [84]:
top_staples_df = staple.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_staples_df

,Symbol,CUM_RET
31,CELH,21.501977
52,HEAR,3.819149
70,KOSS,3.731544
41,DAR,3.583972
56,IMKTA,3.030280
94,NOA,2.989037
116,SMPL,2.751466
66,KDP,2.620130
54,HSY,2.294722
38,COKE,2.264875


In [ ]:
# DAR, KDP, GIS, HSY
for ticker in top_staples_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Discretionary

In [86]:
top_discretion_df = discretion.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_discretion_df

,Symbol,CUM_RET
658,MOBQ,20.000001
1060,XPEL,11.480865
235,CROX,7.365117
154,BXC,7.042296
156,BYRN,6.723077
644,MGNI,6.680473
967,TSLA,6.649895
88,AXON,6.271852
333,ETSY,6.081849
753,PLUG,5.890756


In [ ]:
# TRMD, DDS
for ticker in top_discretion_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Utilities

In [88]:
top_utility_df = utility.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()

In [ ]:
# MUSA, LNG, CWST
for ticker in top_utility_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Finance

In [104]:
top_finance_df = finance.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_finance_df

,Symbol,CUM_RET
118,ATLC,9.487705
650,FRHC,7.466250
1409,PMTS,6.898877
797,HGBL,6.375000
979,KNSL,6.244824
78,AMRK,5.365638
1073,MCVT,4.874423
378,COOP,4.468202
97,ARES,4.467076
677,FUTU,4.338773


In [ ]:
# KNSL, LPLA, AJG
for ticker in top_finance_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Materials

In [106]:
top_material_df = material.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_material_df

,Symbol,CUM_RET
40,PLL,3.276257
28,IDR,2.887755
34,MP,2.861000
16,GFI,2.832208
12,DRD,2.792001
42,RFP,2.531838
44,SBSW,2.481960
14,FCX,2.336194
43,RIO,2.220495
49,UFPI,2.179709


In [ ]:
# RFP, GPK
for ticker in top_material_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Real Estate

In [12]:
top_restate_df = restate.sort_values(by=['CUM_RET'], ascending=False).head(20).copy()
top_restate_df

,Symbol,CUM_RET
217,RCA,41.683332
255,SQFTP,8.515705
206,PECO,5.842574
136,IIPR,5.441114
51,CHCI,3.007692
2,ABR,2.468535
45,BRT,2.437170
143,IRT,2.132455
279,VICI,2.123277
147,JOE,2.038088


In [ ]:
# VICI, IRM
for ticker in top_restate_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Top Energy

In [16]:
top_energy_df = energy.sort_values(by=['CUM_RET'], ascending=False).head(40).tail(20).copy()
top_energy_df

,Symbol,CUM_RET
150,REPX,3.126697
190,VOC,3.040961
38,CRC,2.941711
157,SBR,2.748186
121,NRP,2.657685
185,VAL,2.542616
41,CRT,2.506134
169,SUN,2.482542
48,DKL,2.466621
36,COP,2.464606


In [ ]:
# SBR, SUN, EQNR
for ticker in top_energy_df["Symbol"]:
    print(ticker)
    df_hc = get_stock_df_from_csv(ticker)
    get_Ichimoku(df_hc)

## Selected stocks

In [ ]:
# Analysis done in 17.12.22
stock_list = [
     "SGML", "PWR", "SBR", "SUN", "EQNR", "VICI", "IRM", "RFP", "GPK", "KNSL", "LPLA", "AJG", "MUSA", "LNG", "CWST", "TRMD", "DDS", "DAR", "KDP", "GIS", "HSY", "EXTR", "PTNR", "ENPH", "LSCC", "RENN", "NSSC", "DXCM", "LLY", "CYTK"
]